In [ ]:
#clean files and load into dynamo db

# process the data lines only and keep those which pass the grep with nothing
#grep -e 'CTGTCTCTTATACACATCT' -e 'AAAGAGTGT' -e 'TATGCAGT' -e 'GCTCATGA' -e 'TCATGAGC' -e 'ACACGTCTGAA' -e 'CGTCGTGTAGGG' -e 'GATCGGAAGAGC'

In [41]:

class Adapter_Clean:
    def __init__(self):
        self.num_read=0
        self.N_pos_dict={}
        self.nuc_dict={}
        self.low_phred={}
        self.adapters = {'CTGTCTCTTATACACATCT':0, 'GATCGGAAGAGC':0, 'CTCCAGTCAC':0, 'CGTCGTGTAGGG':0, 'ACACGTCTGAA':0, 'AAAGAGTGT':0,
           'TCATGAGC':0, 'GCTCATGA':0, 'TATGCAGT':0}
        self.PATH="/home/dc/Downloads/"
        self.first_file="empty_S316_L008_R1_001.fastq.gz"
        self.second_file = "empty_S316_L008_R2_001.fastq.gz"
        self.testheader = b'@K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT@K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT'
        self.decoded_testheader = self.testheader.decode('UTF-8')
        self.output_file = self.PATH+"single_output.fastq.gz"
        self.output_filehandle = gzip.open(self.output_file,'wb+')
       
        
    def process_header(self,line,num_at,adapters):
        """
        process the header and add any multiplexing headers we see to the adpaters dictionary
        input: assumes concatenated header with 2 @ signs like:
        
        """
        print("process_header line:",line)
        header = line.split("@")
        #to debug use the test string and python console
        adapters_first = header[1].split()[-1].split(':')[-1].split('+')
        adapters_second = header[2].split()[-1].split(':')[-1].split('+')
    
        for x in adapters_first:
            if x not in self.adapters:
                print("adding multiplixing adapter:",x)
                adapters[x]=0
        for x in adapters_second:
            if x not in self.adapters:
                print("adding multiplixing adapter:",x)
                adapters[x]=0
                
    def test_process_header(self):
        '''
        TBD
        '''
        process_header()
        
    def process_plus(self,read_line,num_plus):
        """
        does nothing but count plus signs for QC
        """
        #print('process_plus')
        if (read_line.strip()=="++"):
            num_plus+=1
        else:
            print("error in plus sign delimiter")


    def process_DNA(self,line,adapters,save):
        """
        1)find lines with N; add those to dictionary of pos where N is. Use this to generate graph of 
        where N is. 
        2)counts of Nucleotides, GC count. 
        """
        print("process_DNA:")
        #get adapter stats
        for x in adapters:
            print('processing adapter:',x)
            if x in line:
                print('found adapter in read!')
                save=False
                adapters[x] += 1
            
        for i,j in enumerate(list(line)):
            if(j=='N'):
                save=False
                #print("DNA bad pos:",i)
                if not i in N_pos_dict:
                    N_pos_dict[i]=1
                else:
                    N_pos_dict[i] +=1
        #stats for nucleotides
            if not j in nuc_dict:
                nuc_dict[j] = 1
            else:
                nuc_dict[j]+=1
        
        
    def process_phred(self,line,save):
        """
        find lines which have phred<20 and store those line positions in low_phred
        return save=false if phred>20
        """
        phred_good=True
        plot_quality=[ord(x)-33 for x in line]
        for i,j in enumerate (plot_quality):
            if j<20:
                save=False
                phred_good=False
                if not i in low_phred:
                    low_phred[i]=1
                else:
                    low_phred[i] +=1
        if phred_good==False:
            print("phred bad < 20")
        else:
            print('phred good')
        #this plots per line you can accumulate these to produce a heat map
        #print(plot_quality)
        #plt.plot(plot_quality)
        #plt.show()
        #plt.gcf().clear()
        
    def plot_phred(self):
        '''
        plot the phred dicttionary for all data
        not sure how this works? generate png? 
        '''
        
    def merge_single_file(self):
        '''
        create single gz file from first_file,second_file
        '''
        startTime = time.time()
    
        with gzip.open(PATH+first_file,"rb") as fh1, gzip.open(PATH+second_file,"rb") as fh2,gzip.open(single_file, 'wb') as f_out:
            for x,y in zip(fh1,fh2):
                readline1 = x.strip()
                readline2 = x.strip()
                full_read = readline1 + readline2
                #print(full_read)
                f_out.write(full_read)
        f_out.close()
        fh1.close()
        fh2.close()
        print(time.time()-startTime)    
        
    def save(self,list_strings):
        '''
        write save buffer to file
        '''
        for x in list_strings:
            self.output_filehandle.write(x)
            self.output_filehandle.write('\n'.encode("utf-8"))
        
    def process_files_for_stats(self):
        '''
        read all lines and make stats for adapters
        '''
        
        read_line = full_read
        num_at = 0
        num_plus = 0
        save_buffer = []
        save=True
        function_lookup = {0:(self.process_header,read_line,num_at,self.adapters),1:(self.process_DNA,read_line,self.adapters,save),2:(self.process_plus,read_line,num_plus),3:(self.process_phred,read_line,save)}
        with gzip.open(self.PATH+self.first_file,"rb") as fh1, gzip.open(self.PATH+self.second_file,"rb") as fh2:
            for x,y in zip(fh1,fh2):
                readline1 = x.strip() ; readline2 = y.strip()
                read_line = (readline1 + readline2).decode('UTF-8')
                save_buffer.append(readline1+readline2)
                func = function_lookup.get(num_at%4)
                print("read_line:",read_line);print("num_at:",num_at, num_at%4, save)
                if(num_at % 4 ==0):
                    if save==True:
                        self.save(save_buffer)
                    save_buffer = []
                    func[0](read_line,num_at,self.adapters)
                elif(num_at%4==1):
                    func[0](read_line,self.adapters,save)
                elif((num_at%4)==2):
                    func[0](read_line,num_plus)
                else:
                    func[0](read_line,save)
                num_at += 1
        self.output_filehandle.close()
    
    def test_process_files_for_stats(self):
        self.first_file = 'test1.fastq.gz'
        self.second_file = 'test2.fastq.gz'
        a.process_files_for_stats()
        
if __name__=="__main__":
    a = Adapter_Clean()
    #a.process_files_for_stats()
    a.test_process_files_for_stats()
    

read_line: @K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT@K00153:508:HN5HMBBXX:8:1101:1641:1138 2:N:0:AAGANGCA+TATGCAGT
num_at: 0 0 True
process_header line: @K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT@K00153:508:HN5HMBBXX:8:1101:1641:1138 2:N:0:AAGANGCA+TATGCAGT
adding multiplixing adapter: AAGANGCA
read_line: GTTNTGCTGTATAAGTTGAATAGATCTGGTGAAAAAACANANCCTTGNCTTACGCTCCTTTCTATTCTTGTATACCTGCTGAATTCACTTCCTATCCTTACACAGACAGTTTGCTGCCAATAATTGATGATAATCCTAACATCTTTTCTNACNAGAATAGTGTGGCTTTGTNAAAGATACCAGAACAAGGAATGCTCAGAATGTTATTTGAAAGAGCAATTGAAATACAGAAGGAACTATATCTGTGCTTTACAGACTACACAAAAGCATTGGACAAAGTACAGCACAAAGACTTGATAGAG
num_at: 1 1 True
process_DNA:
processing adapter: CTGTCTCTTATACACATCT
processing adapter: GATCGGAAGAGC
processing adapter: CTCCAGTCAC
processing adapter: CGTCGTGTAGGG
processing adapter: ACACGTCTGAA
processing adapter: AAAGAGTGT
processing adapter: TCATGAGC
processing adapter: GCTCATGA
processing adapter: TATGCAGT
processing adapter: AAGANGCA
read

In [13]:
import gzip,time
adapters = {'CTGTCTCTTATACACATCT':0, 'GATCGGAAGAGC':0, 'CTCCAGTCAC':0, 'CGTCGTGTAGGG':0, 'ACACGTCTGAA':0, 'AAAGAGTGT':0,
           'TCATGAGC':0, 'GCTCATGA':0, 'TATGCAGT':0}

PATH="/home/dc/Downloads/"
first_file="empty_S316_L008_R1_001.fastq.gz"
second_file = "empty_S316_L008_R2_001.fastq.gz"
single_file = "/home/dc/Downloads/single.fastq.gz"

startTime = time.time()
read_line = full_read
num_at = 0
num_plus = 0
function_lookup = {0:(process_header,read_line,num_at,adapters),1:(process_DNA,read_line),2:(process_plus,read_line,num_plus),3:(process_phred,read_line)}

with gzip.open(PATH+first_file,"rb") as fh1, gzip.open(PATH+second_file,"rb") as fh2,gzip.open(single_file, 'wb') as f_out:
    for x,y in zip(fh1,fh2):
        readline1 = x.strip()
        readline2 = x.strip()
        full_read = readline1 + readline2
        #print(full_read)
        #f_out.write(full_read)
        read_line = full_read.decode('UTF-8')
        func = function_lookup.get(num_read%4)
        if(num_at % 4 ==0):
            func[0](read_line,num_at,adapters)
        num_at += 1
    f_out.close()
    fh1.close()
    fh2.close()
print(time.time()-startTime)    

header, header[0]: K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1641:1138 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1803:1138 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1803:1138 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1824:1138 1:N:0:AAGANGCA+TATGCCGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1824:1138 1:N:0:AAGANGCA+TATGCCGT
adapters_first ['AAGANGCA', 'TATGCCGT']
adapters_second: ['AAGANGCA', 'TATGCCGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1732:1156 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:1732:1156 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K0015

adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10115:1156 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10115:1156 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10135:1156 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10135:1156 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10155:1156 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10155:1156 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second: ['AAGANGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10196:1156 1:N:0:AAGANGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:10196:1156 1:N:0:AAGANGCA+TATGCAGT
adapters_first ['AAGANGCA', 'TATGCAGT']
adapters_second:

adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8542:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8542:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8582:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8582:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8603:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8603:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8643:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8643:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAG

header, header[0]: K00153:508:HN5HMBBXX:8:1101:16173:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16193:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16193:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16214:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16214:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16275:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16275:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16295:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0

adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24190:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24190:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24211:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24211:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24231:1173 1:N:0:AAGAGGCA+TATGCAGC
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24231:1173 1:N:0:AAGAGGCA+TATGCAGC
adapters_first ['AAGAGGCA', 'TATGCAGC']
adapters_second: ['AAGAGGCA', 'TATGCAGC']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24312:1173 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:24312:1173 1:N:0:AAGAGGCA+TATGCAGT
adapters_first [

...

adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8075:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8075:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8095:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8095:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8115:1244 1:N:0:CAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8115:1244 1:N:0:CAGAGGCA+TATGCAGT
adapters_first ['CAGAGGCA', 'TATGCAGT']
adapters_second: ['CAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8136:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:8136:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAG

header, header[0]: K00153:508:HN5HMBBXX:8:1101:16376:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16376:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16396:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16396:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16417:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16417:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16437:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:16437:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0

header, header[0]: K00153:508:HN5HMBBXX:8:1101:25875:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25895:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25895:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25915:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25915:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25956:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25956:1244 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:25976:1244 1:N:0:AAGAGGCA+TATGCAGT
header, header[0

header, header[0]: K00153:508:HN5HMBBXX:8:1101:8998:1261 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9019:1261 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9019:1261 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9039:1261 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9039:1261 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9100:1261 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9100:1261 1:N:0:AAGAGGCA+TATGCAGT
adapters_first ['AAGAGGCA', 'TATGCAGT']
adapters_second: ['AAGAGGCA', 'TATGCAGT']
header, header[0]: K00153:508:HN5HMBBXX:8:1101:9161:1261 1:N:0:AAGAGGCA+TATGCAGT
header, header[0]: K0015

KeyboardInterrupt: 